In [ ]:
import pandas as pd
import csv
from sqlalchemy import create_engine
# Build Supabase engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)

replace = pd.read_csv("item name replace.csv")

#SO
SO_INV = pd.read_sql_table("wo_structured", con=engine, schema="public")
SO = SO_INV[['Order Date', 'Ship Date', 'QB Num', "P. O. #", "Name",'Qty(+)', 'Qty(-)', 'Item', 'Pre/Bare', 'On Hand', "On Hand - WIP"]]
# SO.to_csv('open sales2.csv',index=False,columns =SO)

#"POD"
pod = pd.read_sql_table("Open_Purchase_Orders", con=engine, schema="public")
# pod.to_csv('open purchase2.csv', index=False)

In [5]:
SO.head(3)

,Order Date,Ship Date,QB Num,P. O. #,Name,Qty(+),Qty(-),Item,Pre/Bare,On Hand,On Hand - WIP
0,2025/08/22,2099/12/31,EO-20250002,P96695,"CoastIPC, Inc.",0,1.0,RGS-8805GC,Out,0.0,0.0
1,2024/03/13,2025/12/19,SO-20240315,X110992,Xanthon LLC,0,1.0,POC-715,Out,31.0,31.0
2,2024/03/13,2025/12/19,SO-20240315,X110992,Xanthon LLC,0,1.0,DDR5-16GB-WT48-IK,Out,48.0,48.0


In [49]:
NAV= pd.read_sql_table("NT Shipping Schedule", con=engine, schema="public")
NAV.to_csv('NAV1.csv', index=False)

# 讀取 NAV1 並篩選符合條件的數據
s50 = []

for _, row in NAV.iterrows():
    if row['Pre/Bare'] == 'Pre':
        s50.append(row)

s50 = pd.DataFrame(s50)

s50

,SO NO.,QB Num,Item,Description,Ship Date,Qty(+),Pre/Bare
6,SO25090098,POD-251261,SEMIL-2047GC-CRL-CF03,"SEMIL-2047GC-CRL, including i9-13900E, 2x SSD-...",2025/10/22,3,Pre
39,SO25090386,POD-251346,Nuvo-9531-PoE-i9IC,"Nuvo-9531-PoE, including i9-13900TE",2025/10/23,3,Pre
41,SO25090525,POD-251385,NRU-52S-NX16G,"NRU-52S-NX, including GC-Jetson-NX16GB-Xavier...",2025/10/22,3,Pre
44,SO25100015,POD-251395,NRU-230V-AWP-JAO64G,"NRU-230V-AWP, including GC-Jetson-AGX64GB-Ori...",2025/10/22,8,Pre
47,SO25100017,POD-251407,N7162GCPoE-i7G9-65W-P22-PaR01-1,"Nuvo-7162GC-PoE, including GC-P2200-Leadtek",2025/10/22,1,Pre
...,...,...,...,...,...,...,...
343,SO25100306,POD-251459,NRU-240S-AWP-JAO64G,"NRU-240S-AWP, including GC-Jetson-AGX64GB-Ori...",None,1,Pre
344,SO25100306,POD-251459,Nuvo-9531-PoE-IGN-i5HC,"Nuvo-9531-PoE-IGN, including i5-12500TE",None,4,Pre
345,SO25100306,POD-251459,Nuvo-9160GC-PoE-i5HC,"Nuvo-9160GC-PoE, I226 version, including i5-1...",None,2,Pre
346,SO25100306,POD-251459,Nuvo-9006E-UL-i5HC-65W,"Nuvo-9006E-UL-i226, I226 version, including i...",None,4,Pre


In [54]:
result_lists = []
for _, original_list in s50.iterrows():
    # 分割字串
    product_str = original_list['Description']
    product_str = product_str.replace('\u00A0', ' ').replace('\u3000', ' ')
    product_info = product_str.split(', including ')
    #product_info = original_list[-1].split(', including ')
    product_info[0] = product_info[0].split(',')[0]  # 產品代碼
    components = product_info[1].split(', ') if len(product_info) > 1 else []
    # 建立各組件的新 list
    for component in components:
        new_list = original_list.copy()
        new_list['Item'] = component.strip()
        result_lists.append(new_list)

    # 加入產品代碼(Machine)
    new_list_with_product_code = original_list.copy()
    new_list_with_product_code['Item'] = product_info[0]
    result_lists.append(new_list_with_product_code)
result_lists

[SO NO.                                                SO25090098
 QB Num                                                POD-251261
 Item                                                   i9-13900E
 Description    SEMIL-2047GC-CRL, including i9-13900E, 2x SSD-...
 Ship Date                                             2025/10/22
 Qty(+)                                                         3
 Pre/Bare                                                     Pre
 Name: 6, dtype: object,
 SO NO.                                                SO25090098
 QB Num                                                POD-251261
 Item                                        2x SSD-2TB-TLC5WT-TD
 Description    SEMIL-2047GC-CRL, including i9-13900E, 2x SSD-...
 Ship Date                                             2025/10/22
 Qty(+)                                                         3
 Pre/Bare                                                     Pre
 Name: 6, dtype: object,
 SO NO.                   

In [56]:
# for i in range(0, len(result_lists)):
#     result_lists
#     # result_lists[i][3] = result_lists[i][6]
#     result_lists[i]['Item'] = result_lists[i]['Description']
    
# 調整數據格式
transformed_lists = []
for result_list in result_lists:
    transformed_list = result_list.copy()
    transformed_list[3] = transformed_list[3].replace(" ", "")
    
    if len(transformed_list[3]) > 1 and transformed_list[3][1] == 'x' and transformed_list[3][0].isdigit():
        quantity = int(transformed_list[3].split('x')[0])
        name = transformed_list[3].split('x')[-1]
        transformed_list[3] = name
        transformed_list[5] = str(quantity * float(transformed_list[5]))  # 更新數量

    transformed_lists.append(transformed_list)
transformed_lists

C:\Users\Admin\AppData\Local\Temp\ipykernel_16956\1542251013.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  transformed_list[3] = transformed_list[3].replace(" ", "")
C:\Users\Admin\AppData\Local\Temp\ipykernel_16956\1542251013.py:10: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  transformed_list[3] = transformed_list[3].replace(" ", "")
C:\Users\Admin\AppData\Local\Temp\ipykernel_16956\1542251013.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

[SO NO.                                                SO25090098
 QB Num                                                POD-251261
 Item                                                   i9-13900E
 Description    SEMIL-2047GC-CRL,includingi9-13900E,2xSSD-2TB-...
 Ship Date                                             2025/10/22
 Qty(+)                                                         3
 Pre/Bare                                                     Pre
 Name: 6, dtype: object,
 SO NO.                                                SO25090098
 QB Num                                                POD-251261
 Item                                        2x SSD-2TB-TLC5WT-TD
 Description    SEMIL-2047GC-CRL,includingi9-13900E,2xSSD-2TB-...
 Ship Date                                             2025/10/22
 Qty(+)                                                         3
 Pre/Bare                                                     Pre
 Name: 6, dtype: object,
 SO NO.                   

In [ ]:
# 追加寫入 NAV1
with open('NAV1.csv', 'a+', encoding='utf-8', newline="") as csvfile:
    write = csv.writer(csvfile)
    write.writerows(transformed_lists)

# # NAV 加上倉別和日期
# # NAV = pd.read_csv("NAV1.csv", usecols=['Remark', 'QB Num', 'Item', 'Qty(+)', 'Ship Date'], encoding='utf-8')
# NAV = NAV[['QB Num', 'Item', 'Qty(+)', 'Ship Date']]
# replace_dict = dict(zip(replace['NAV'], replace['QB']))
# NAV['Item'] = NAV['Item'].replace(replace_dict)
# NAV.to_csv('NAV1.csv', index=False)